In [1]:
from __future__ import division
%pylab inline
from scipy import stats
from angles import r2d, r2arcs, d2arcs, arcs2r
def arcm2r(theta):
    return arcs2r(theta*60)
import seaborn as sns;sns.set_style('darkgrid')
import lsst.sims.maf.stackers as stackers
import treecorr as tr
%config InlineBackend.figure_format = 'retina'
matplotlib.rcParams['figure.figsize'] = 19,14
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.metricBundles as metricBundles
import lsst.sims.maf.db as db
from collections import defaultdict
import treecorr
from itertools import chain
sns.set_context('poster',font_scale=1.4)

Populating the interactive namespace from numpy and matplotlib


In [ ]:
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.batches as batches
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.metricBundles as metricBundles
from scipy.stats import uniform, kstest

In [ ]:
def compass_bearing(lon1, lat1, lon2, lat2):
    """Calculate the bearing between two points
    
    Parameters
    ----------
    lon1 : float (or array)
        The longitude(s) for starting point(s) (radians)
    lat1 : float (or arrray)
        The latitudes(s) for the starting point(s) (radians)
    lon2 : float (or array)
        The longitude for the ending point(s) (radians)
    lat2 : float (or array)
        The latitude for the ending point(s) (radians)
        
    Returns
    -------
    bearing in radians (between 0 and 2pi)
        
    """
    long_diff = lon2 - lon1
    x = np.sin(long_diff)*np.cos(lat2)
    y = np.cos(lat1)*np.sin(lat2)-np.sin(lat1)*np.cos(lat2)*np.cos(long_diff)
    result = np.arctan2(x, y)
    result = (result+2*np.pi) % (2.*np.pi)
    return result



In [ ]:
test_positions = {}
class AngleDist2PointingCenter(metrics.BaseMetric):
    """See how flat the distributing of angles pointing to the center of the FoV is
    
    Uses the KStest statistic--values of 0 are perfectly uniform, values of 1 are totally non-uniform.
    """
    def __init__(self, latCol='dec', lonCol='RA', degrees=True,
                 metricName='Pointing Center Distribution', **kwargs):
        # Make a uniform distribution beween 0 and 2pi
        col = [latCol, lonCol]
        self.latCol = latCol
        self.lonCol = lonCol
        super(AngleDist2PointingCenter, self).__init__(col=col, units='KS Statistic',
                                                       metricName=metricName, **kwargs)
        self.dist = uniform(loc=0, scale=2.*np.pi)
        self.degrees = degrees
    def run(self, dataSlice, slicePoint=None):
        if self.degrees:
            lat2 = np.radians(dataSlice[self.latCol])
            lon2 = np.radians(dataSlice[self.lonCol])
        else:
            lat2 = dataSlice[self.latCol]
            lon2 = dataSlice[self.lonCol]
        global test_positions
        test_positions['lon2, lat2'] = (lon2, lat2)
        test_positions['slicepoint'] = (slicePoint['ra'], slicePoint['dec'])
        # Bearing from this point in the sky to all the pointing centers
        bearings = compass_bearing(slicePoint['ra'], slicePoint['dec'], lon2, lat2)
        ks_result = kstest(bearings, self.dist.cdf)
        return ks_result.pvalue



### Random

In [ ]:
outDir='temp'
conn = db.OpsimDatabaseV3('simruns/minion_1012_sqlite.db')
bundleList = []
sql = 'filter = "r" and night < 3653 \
and fieldRA < {} and fieldDec > {} \
and fieldRA > 0 and fieldDec < 0'.format(np.radians(200), np.radians(-50))
metric = AngleDist2PointingCenter(latCol='randomDitherFieldPerVisitDec', lonCol='randomDitherFieldPerVisitRa', degrees=False)
slicer = slicers.HealpixSlicer(nside=128, latCol='randomDitherFieldPerVisitDec', lonCol='randomDitherFieldPerVisitRa', latLonDeg=False)
dith_stack = stackers.RandomDitherFieldPerVisitStacker(raCol='fieldRA', decCol='fieldDec', degrees=False)
bundleList.append(metricBundles.MetricBundle(metric, slicer, sql, stackerList=[dith_stack]))
bd = metricBundles.makeBundlesDictFromList(bundleList)
mbg = metricBundles.MetricBundleGroup(bd, conn, outDir=outDir)
mbg.runAll()


results_bundle = bd['opsim_Pointing_Center_Distribution_r_and_night_lt_3653_and_fieldRA_lt_3_490658503988659_and_fieldDec_gt_-0_8726646259971648_and_fieldRA_gt_0_and_fieldDec_lt_0_HEAL']

random_hist = results_bundle.metricValues.data[np.where(results_bundle.metricValues.mask==False)]

### Hexagonal

In [ ]:
outDir='temp'
conn = db.OpsimDatabaseV3('simruns/minion_1012_sqlite.db')
bundleList = []
sql = 'filter = "r" and night < 3653 \
and fieldRA < {} and fieldDec > {} \
and fieldRA > 0 and fieldDec < 0'.format(np.radians(200), np.radians(-50))
metric = AngleDist2PointingCenter(latCol='hexDitherFieldPerVisitDec', lonCol='hexDitherFieldPerVisitRa', degrees=False)
slicer = slicers.HealpixSlicer(nside=128, latCol='hexDitherFieldPerVisitDec', lonCol='hexDitherFieldPerVisitRa', latLonDeg=False)
dith_stack = stackers.HexDitherFieldPerVisitStacker(raCol='fieldRA', decCol='fieldDec', degrees=False, fieldIdCol='fieldID')
bundleList.append(metricBundles.MetricBundle(metric, slicer, sql, stackerList=[dith_stack]))
bd = metricBundles.makeBundlesDictFromList(bundleList)
mbg = metricBundles.MetricBundleGroup(bd, conn, outDir=outDir)
mbg.runAll()
conn.close()


results_bundle = bd['opsim_Pointing_Center_Distribution_r_and_night_lt_3653_and_fieldRA_lt_3_490658503988659_and_fieldDec_gt_-0_8726646259971648_and_fieldRA_gt_0_and_fieldDec_lt_0_HEAL']

hex_hist = results_bundle.metricValues.data[np.where(results_bundle.metricValues.mask==False)]

### Spiral

In [ ]:
outDir='temp'
conn = db.OpsimDatabaseV3('simruns/minion_1012_sqlite.db')
bundleList = []
sql = 'filter = "r" and night < 3653 \
and fieldRA < {} and fieldDec > {} \
and fieldRA > 0 and fieldDec < 0'.format(np.radians(200), np.radians(-50))
metric = AngleDist2PointingCenter(latCol='spiralDitherFieldPerVisitDec', lonCol='spiralDitherFieldPerVisitRa', degrees=False)
slicer = slicers.HealpixSlicer(nside=128, latCol='spiralDitherFieldPerVisitDec', lonCol='spiralDitherFieldPerVisitRa', latLonDeg=False)
dith_stack = stackers.SpiralDitherFieldPerVisitStacker(raCol='fieldRA', decCol='fieldDec', degrees=False, fieldIdCol='fieldID')
bundleList.append(metricBundles.MetricBundle(metric, slicer, sql, stackerList=[dith_stack]))
bd = metricBundles.makeBundlesDictFromList(bundleList)
mbg = metricBundles.MetricBundleGroup(bd, conn, outDir=outDir)
mbg.runAll()
conn.close()


results_bundle = bd['opsim_Pointing_Center_Distribution_r_and_night_lt_3653_and_fieldRA_lt_3_490658503988659_and_fieldDec_gt_-0_8726646259971648_and_fieldRA_gt_0_and_fieldDec_lt_0_HEAL']

spiral_hist = results_bundle.metricValues.data[np.where(results_bundle.metricValues.mask==False)]

## Feature Baseline

In [ ]:
outDir='temp'
conn = db.OpsimDatabaseV4('simruns/feature_baseline_update_10yrsv2')
resultsDb = db.ResultsDb(outDir=outDir)

bundleList = []
sql = 'filter = "r" and night < 3653 \
and fieldRA < {} and fieldDec > {} \
and fieldRA > 0 and fieldDec < 0 and proposalId = 0'.format(200,-50)
metric = AngleDist2PointingCenter(latCol='fieldDec', lonCol='fieldRA')
slicer = slicers.HealpixSlicer(nside=128, latCol='fieldDec', lonCol='fieldRA')
bundleList.append(metricBundles.MetricBundle(metric, slicer, sql))
bd = metricBundles.makeBundlesDictFromList(bundleList)
mbg = metricBundles.MetricBundleGroup(bd, conn, outDir=outDir)
mbg.runAll()
conn.close()

results_bundle = bd['opsim_Pointing_Center_Distribution_r_and_night_lt_3653_and_fieldRA_lt_200_and_fieldDec_gt_-50_and_fieldRA_gt_0_and_fieldDec_lt_0_and_proposalId_0_HEAL']

feature_hist = results_bundle.metricValues.data[np.where(results_bundle.metricValues.mask==False)]

### ALT_Sched

In [ ]:
outDir='temp'
conn = db.OpsimDatabaseV4('simruns/alt_sched.db')
resultsDb = db.ResultsDb(outDir=outDir)

bundleList = []
sql = 'filter = "r" and night < 3653 \
and fieldRA < {} and fieldDec > {} \
and fieldRA > 0 and fieldDec < 0 and proposalId = 0'.format(200,-50)
metric = AngleDist2PointingCenter(latCol='fieldDec', lonCol='fieldRA')
slicer = slicers.HealpixSlicer(nside=128, latCol='fieldDec', lonCol='fieldRA')
bundleList.append(metricBundles.MetricBundle(metric, slicer, sql))
bd = metricBundles.makeBundlesDictFromList(bundleList)
mbg = metricBundles.MetricBundleGroup(bd, conn, outDir=outDir)
mbg.runAll()
conn.close()

results_bundle = bd['opsim_Pointing_Center_Distribution_r_and_night_lt_3653_and_fieldRA_lt_200_and_fieldDec_gt_-50_and_fieldRA_gt_0_and_fieldDec_lt_0_and_proposalId_0_HEAL']

alt_hist = results_bundle.metricValues.data[np.where(results_bundle.metricValues.mask==False)]

In [ ]:
plt.hist(random_hist, label='random', bins=100, alpha=0.5)
plt.hist(spiral_hist, label='spiral', bins=100, alpha=0.5)
plt.hist(hex_hist, label='hex', bins=100, alpha=0.5)
plt.hist(alt_hist, label='alt_sched', bins=100, alpha=0.5)
plt.hist(feature_hist, label='featureBaseline', bins=100, alpha=0.5)


plt.xscale('log')
plt.title('D-Statistic')
plt.legend()

In [ ]:
outDir='temp'
conn = db.OpsimDatabaseV3('simruns/minion_1012_sqlite.db')
bundleList = []
sql = 'filter = "r" and night < 3653 \
and fieldRA < {} and fieldDec > {} \
and fieldRA > 0 and fieldDec < 0'.format(np.radians(200), np.radians(-50))
metric = metrics.PassMetric()
slicer = slicers.OneDSlicer(sliceColName='night', binsize=1)
dith_stack = stackers.RandomRotDitherPerFilterChangeStacker( degrees=False)
bundleList.append(metricBundles.MetricBundle(metric, slicer, sql, stackerList=[dith_stack]))
bd = metricBundles.makeBundlesDictFromList(bundleList)
mbg = metricBundles.MetricBundleGroup(bd, conn, outDir=outDir, saveEarly= False)
mbg.runAll()
mbg.plotAll(closefigs=False)
conn.close()


